In [24]:
%load_ext autoreload
%autoreload 2
from functools import partial

import keras

from datasets import example_datasets, to_numpy
from models import mixture_poissons,poisson_glm
from metrics import mixture_poi_loss, get_bpr_loss_func, mix_bpr, get_penalized_bpr_loss_func_mix
from experiments import training_loop
from plotting_funcs import plot_losses, plot_frontier

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
seed=360
num_components=4
learning_rate = 0.005
epochs=1250
outdir = '/cluster/home/kheuto01/testdir'
penalty = 0
threshold = 0.45
K=4
do_only=True
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 500

In [15]:
train_dataset, val_dataset, test_dataset = example_datasets(H, T, seed=seed)

input_shape = (H,S)

negative_bpr_K = get_bpr_loss_func(K)

In [16]:
mix_model, mix_weights  = mixture_poissons(poisson_glm, input_shape, num_components=num_components)

In [30]:
mix_model.variables

[<tf.Variable 'conv1d_8/kernel:0' shape=(1, 3, 1) dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Variable 'conv1d_8/bias:0' shape=(1,) dtype=float32, numpy=array([-inf], dtype=float32)>,
 <tf.Variable 'conv1d_9/kernel:0' shape=(1, 3, 1) dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Variable 'conv1d_9/bias:0' shape=(1,) dtype=float32, numpy=array([-inf], dtype=float32)>,
 <tf.Variable 'conv1d_10/kernel:0' shape=(1, 3, 1) dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Variable 'conv1d_10/bias:0' shape=(1,) dtype=float32, numpy=array([-inf], dtype=float32)>,
 <tf.Variable 'conv1d_11/kernel:0' shape=(1, 3, 1) dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Variable 'conv1d_11/bias:0' shape=(1,) dtype=float32, numpy=array([-inf], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4, 12) dtype=float32, numpy=
 array([[

Component 1-4 Weights (all zero)

In [28]:
for index in [0, 2, 4, 6]:
    variable = mix_model.trainable_variables[index]
    print(f'Name: {variable.name}')
    print(f'Shape: {variable.shape}')

    variable.assign(tf.zeros(variable.shape))

Name: conv1d_8/kernel:0
Shape: (1, 3, 1)
Name: conv1d_9/kernel:0
Shape: (1, 3, 1)
Name: conv1d_10/kernel:0
Shape: (1, 3, 1)
Name: conv1d_11/kernel:0
Shape: (1, 3, 1)


Component 1-4 Biases, [0, 7, 10, 100]

In [75]:
for index, bias in zip([1, 3, 5, 7], [0.0+1e-13, 7.0, 10.0, 100.0]):
    variable = mix_model.trainable_variables[index]
    print(f'Name: {variable.name}')
    print(f'Shape: {variable.shape}')

    variable.assign(tfp.math.softplus_inverse([bias]))

Name: conv1d_8/bias:0
Shape: (1,)
Name: conv1d_9/bias:0
Shape: (1,)
Name: conv1d_10/bias:0
Shape: (1,)
Name: conv1d_11/bias:0
Shape: (1,)


Mixture Weights

In [76]:
index = -1
variable = mix_model.trainable_variables[index]
print(f'Name: {variable.name}')
print(f'Shape: {variable.shape}')

# equal to 0
component_1 = np.array([0,0,0,0, 0.3, 0.3, 0.3, 0.3, 0.9, 0.9, 0.9, 0.9], dtype=np.float32)
# equal to 7
component_2 = np.array([1,1,1,1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32)
# equal to 10
component_3 = np.array([0,0,0,0, 0.7, 0.7, 0.7, 0.7, 0, 0, 0, 0], dtype=np.float32)
# equal to 10
component_4 = np.array([0,0,0,0, 0,0,0,0, 0.1, 0.1, 0.1, 0.1], dtype=np.float32)

weights = np.vstack([component_1, component_2, component_3, component_4])
variable.assign(tf.math.log(weights+1e-13))

Name: Variable:0
Shape: (4, 12)


<tf.Variable 'UnreadVariable' shape=(4, 12) dtype=float32, numpy=
array([[-29.933605  , -29.933605  , -29.933605  , -29.933605  ,
         -1.2039728 ,  -1.2039728 ,  -1.2039728 ,  -1.2039728 ,
         -0.10536054,  -0.10536054,  -0.10536054,  -0.10536054],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        -29.933605  , -29.933605  , -29.933605  , -29.933605  ,
        -29.933605  , -29.933605  , -29.933605  , -29.933605  ],
       [-29.933605  , -29.933605  , -29.933605  , -29.933605  ,
         -0.35667497,  -0.35667497,  -0.35667497,  -0.35667497,
        -29.933605  , -29.933605  , -29.933605  , -29.933605  ],
       [-29.933605  , -29.933605  , -29.933605  , -29.933605  ,
        -29.933605  , -29.933605  , -29.933605  , -29.933605  ,
         -2.3025851 ,  -2.3025851 ,  -2.3025851 ,  -2.3025851 ]],
      dtype=float32)>

In [77]:
mix_model.variables

[<tf.Variable 'conv1d_8/kernel:0' shape=(1, 3, 1) dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Variable 'conv1d_8/bias:0' shape=(1,) dtype=float32, numpy=array([-29.933607], dtype=float32)>,
 <tf.Variable 'conv1d_9/kernel:0' shape=(1, 3, 1) dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Variable 'conv1d_9/bias:0' shape=(1,) dtype=float32, numpy=array([6.999088], dtype=float32)>,
 <tf.Variable 'conv1d_10/kernel:0' shape=(1, 3, 1) dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Variable 'conv1d_10/bias:0' shape=(1,) dtype=float32, numpy=array([9.999954], dtype=float32)>,
 <tf.Variable 'conv1d_11/kernel:0' shape=(1, 3, 1) dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Variable 'conv1d_11/bias:0' shape=(1,) dtype=float32, numpy=array([100.], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4, 12) dtype=float32, nu

In [78]:
train_X_THS, train_y_TS = to_numpy(train_dataset)
val_X_THS, val_y_TS = to_numpy(val_dataset)

In [79]:
y_preds = mix_model(train_X_THS)

In [80]:
negative_bpr_K = get_bpr_loss_func(K)

In [81]:
nll = mixture_poi_loss(train_y_TS, y_preds)
bpr = mix_bpr(train_y_TS, y_preds, negative_bpr_K_func=negative_bpr_K)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got partial
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got partial
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [82]:
tf.reduce_mean(bpr)

<tf.Tensor: shape=(), dtype=float32, numpy=-0.27111065>

In [83]:
y_preds[1]

<tf.Tensor: shape=(4, 12), dtype=float32, numpy=
array([[1.00000100e-13, 1.00000100e-13, 1.00000100e-13, 1.00000100e-13,
        2.99999982e-01, 2.99999982e-01, 2.99999982e-01, 2.99999982e-01,
        9.00000036e-01, 9.00000036e-01, 9.00000036e-01, 9.00000036e-01],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000032e-13, 1.00000032e-13, 1.00000032e-13, 1.00000032e-13,
        1.00000066e-13, 1.00000066e-13, 1.00000066e-13, 1.00000066e-13],
       [1.00000100e-13, 1.00000100e-13, 1.00000100e-13, 1.00000100e-13,
        6.99999988e-01, 6.99999988e-01, 6.99999988e-01, 6.99999988e-01,
        1.00000066e-13, 1.00000066e-13, 1.00000066e-13, 1.00000066e-13],
       [1.00000100e-13, 1.00000100e-13, 1.00000100e-13, 1.00000100e-13,
        1.00000032e-13, 1.00000032e-13, 1.00000032e-13, 1.00000032e-13,
        9.99999940e-02, 9.99999940e-02, 9.99999940e-02, 9.99999940e-02]],
      dtype=float32)>

In [84]:
component_preds, mixture_weights = y_preds

mixture_pred = tf.einsum('ijk,kj->ij', component_preds, mixture_weights)

In [85]:
mixture_pred[0]

<tf.Tensor: shape=(12,), dtype=float32, numpy=
array([7.      , 7.      , 7.      , 7.      , 7.      , 7.      ,
       7.      , 7.      , 9.999999, 9.999999, 9.999999, 9.999999],
      dtype=float32)>

In [86]:
mixture_weights

<tf.Tensor: shape=(4, 12), dtype=float32, numpy=
array([[1.00000100e-13, 1.00000100e-13, 1.00000100e-13, 1.00000100e-13,
        2.99999982e-01, 2.99999982e-01, 2.99999982e-01, 2.99999982e-01,
        9.00000036e-01, 9.00000036e-01, 9.00000036e-01, 9.00000036e-01],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000032e-13, 1.00000032e-13, 1.00000032e-13, 1.00000032e-13,
        1.00000066e-13, 1.00000066e-13, 1.00000066e-13, 1.00000066e-13],
       [1.00000100e-13, 1.00000100e-13, 1.00000100e-13, 1.00000100e-13,
        6.99999988e-01, 6.99999988e-01, 6.99999988e-01, 6.99999988e-01,
        1.00000066e-13, 1.00000066e-13, 1.00000066e-13, 1.00000066e-13],
       [1.00000100e-13, 1.00000100e-13, 1.00000100e-13, 1.00000100e-13,
        1.00000032e-13, 1.00000032e-13, 1.00000032e-13, 1.00000032e-13,
        9.99999940e-02, 9.99999940e-02, 9.99999940e-02, 9.99999940e-02]],
      dtype=float32)>

In [87]:
y_preds[0][0]

<tf.Tensor: shape=(12, 4), dtype=float32, numpy=
array([[9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02],
       [9.99999e-14, 7.00000e+00, 1.00000e+01, 1.00000e+02]],
      dtype=float32)>